In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
output_data_file = "output_data/cities.csv"

In [4]:
#Load the csv exported in Part I to a DataFrame

cities_df = pd.read_csv(output_data_file)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind
0,camana,52.0,PE,1.580174e+09,83.0,-16.62,-72.71,68.90,5.26
1,pevek,64.0,RU,1.580174e+09,88.0,69.70,170.31,-3.51,3.65
2,puerto ayora,75.0,EC,1.580174e+09,61.0,-0.74,-90.35,82.40,4.70
3,vaini,75.0,TO,1.580174e+09,88.0,-21.20,-175.20,80.60,4.70
4,honningsvag,52.0,NO,1.580174e+09,64.0,70.98,25.97,22.19,18.21


In [5]:
# Drop NA values from dataframe
cities_clean = cities_df.dropna(how='any')

# Confirm drop
cities_clean.count()

City               555
Cloudiness         555
Country            555
Date               555
Humidity           555
Lat                555
Lng                555
Max Temperature    555
Wind               555
dtype: int64

In [6]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

gmaps.configure(api_key=g_key)

locations = cities_clean[["Lat", "Lng"]].astype(float)
humidity = cities_clean['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create new DataFrame fitting weather criteria
# Country is Mexico
# Humidity less than 50
# Wind > 0

mx_cities = cities_clean[(cities_clean.Country == 'MX')]
mx_cities_humid = mx_cities[(mx_cities.Humidity <= 50)]
mx_vacation_cities = mx_cities_humid[(mx_cities_humid.Wind > 0)]
mx_vacation_cities.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind
211,ojinaga,75.0,MX,1.580174e+09,24.0,29.57,-104.42,71.60,9.17
546,camargo,0.0,MX,1.580174e+09,19.0,27.67,-105.17,71.26,17.81


In [8]:
mx_vacation_cities['Hotel Name'] = ''

for index, row in mx_vacation_cities.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    lat = row['Lat']
    lng = row['Lng']
    
    params = {
    "location": (f'{lat}, {lng}'), 
    "type": "lodging",
    "radius": "5000",
    "key": g_key}
    
    hotel_df = requests.get(base_url, params=params).json()
    
    try:
        mx_vacation_cities.loc[index, "Hotel Name"] = hotel_df['results'][0]['name']
        
    except IndexError:
        
        mx_vacation_cities.loc[index, "Hotel Name"] = 'N/A'

mx_vacation_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind,Hotel Name
211,ojinaga,75.0,MX,1.580174e+09,24.0,29.57,-104.42,71.60,9.17,Suites Las Palmas
546,camargo,0.0,MX,1.580174e+09,19.0,27.67,-105.17,71.26,17.81,Hotel Santa Fe


In [9]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in mx_vacation_cities.iterrows()]
locations = mx_vacation_cities[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(
                locations,
                info_box_content=hotel_info)


fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))